In [1]:
!pip3 install pandas boto3 mysql-connector-python scikit-learn numpy

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier # Importa o classificador K-Nearest Neighbors (KNN)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # Importa métricas de avaliação
import time
import sys
sys.path.append('../..') # Adiciona o diretório superior ao caminho de importação para acessar funções utilitárias
from functions.db_inserts import * # Importa funções para inserção de dados no banco de dados
from functions.aws_functions import * # Importa funções relacionadas à AWS
from functions.select_cols_functions import * # Importa funções para seleção e processamento de colunas

In [3]:
env_mode = 'prd' # Define o modo de ambiente: 'dev' para desenvolvimento, 'prd' para produção

### Lendo arquivo fonte

In [4]:
bucket_name = "adoptai-trusted-dev" # Nome do bucket S3 de onde os dados serão carregados

In [5]:
if env_mode == 'dev':
    # Modo de desenvolvimento: carrega e processa os dados localmente
    df_trusted = pd.read_csv('../../../data/train_2_classes.csv')
    df_trusted = apply_all_rules(df_trusted)
    s3 = ['train_2_classes.csv', df_trusted]
elif env_mode == 'prd':
    # Modo de produção: carrega e processa os dados a partir do S3
    s3 = s3_csv_to_df(bucket_name)
    df_trusted = s3[1]

### Utilizando LabelEncoded nas colunas categóricas

In [6]:
df_trusted = adjust_categoric_values(df_trusted)

### Parâmetros de gravação

In [7]:
nome_fonte = s3[0] # Armazena o nome do arquivo de dados utilizado
nome_modelo = 'KNN' # Define o nome do modelo
parameters = {'k': list(range(1, 20))} # Define os valores de k a serem testados para o modelo KNN

### Preparativos para executar o modelo

In [8]:
# Separar as features (X) e os rótulos (y)
x = df_trusted.iloc[:, :-1] # Seleciona todas as colunas, exceto a última, como features
y = df_trusted.iloc[:, -1] # Seleciona a última coluna como rótulo

In [9]:
# Dividir os dados em conjuntos de treinamento e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

###  Rodando modelo KNN

In [10]:
# Listas para armazenar métricas de desempenho e tempos de execução
accuracy = []
precision = []
recall = []
f1 = []

start_times = []
end_times = []

In [11]:
# Treina e avalia o modelo KNN para diferentes valores de k
for k in parameters['k']:
    start_time = start_times.append(time.time()) # Registra o tempo de início
    knn = KNeighborsClassifier(n_neighbors=k) # Inicializa o modelo KNN com o valor atual de k
    knn.fit(x_train, y_train) # Treina o modelo com os dados de treinamento

    y_pred = knn.predict(x_test) # Fazer previsões no conjunto de teste

    # Avaliar o modelo e armazenar as métricas
    accuracy.append(accuracy_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred, average=None))
    recall.append(recall_score(y_test, y_pred, average=None))
    f1.append(f1_score(y_test, y_pred, average=None))

    end_times.append(time.time()) # Registra o tempo de término

### Obtendo métricas de desempenho

In [12]:
# Armazena as métricas calculadas
metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1
}

### Gravando dados no banco

In [13]:
# Prepara os dados para inserção no banco de dados
model = {'nome_modelo': nome_modelo, 'nome_fonte': nome_fonte}
execution = {'accuracy': accuracy, 'start_time': start_times, 'end_time': end_times}
hyperparams = parameters
desempenho = {'precision_values': metrics['precision'], 'recall_values': metrics['recall'], 'f1_values': metrics['f1_score']}

In [14]:
# Insere os valores calculados nas tabelas do banco de dados
insert_values(model, execution, hyperparams, desempenho, env=env_mode)

Executando em dev
Registros inseridos em dev
